In [14]:
import talib as taLib
import pymongo
import pandas as pd
import numpy as np
import nsepy
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]


8425

In [15]:
def MovingAverageAnalysis(df,periodfast,periodslow):
    df['EWMfast']=taLib.EMA(df['Close'].values,periodfast)
    df['EWMslow']=taLib.EMA(df['Close'].values,periodslow)
    tempnp=df.iloc[-1].to_numpy()
    dayssincecutover=0
    if tempnp[16] < tempnp[15]:
        trend=1
    else:
        trend=-1
    for j in range(df.shape[0]-1,2,-1):
        if (datadf.iloc[j,15]-datadf.iloc[j,16])*(datadf.iloc[j-1,15]-datadf.iloc[j-1,16]) < 0 :
            dayssincecutover=datadf.shape[0]-j
            break;
    (macd, macdsignal, macdhist)=(taLib.MACD(df['Close'], fastperiod=periodfast, slowperiod=periodslow, signalperiod=9))
    tempnp=macdhist.iloc[-1]
    if tempnp > 1:
        trendnature='Expanding'
    else:
        trendnature='Contracting'
    #print(df.iloc[-1])   
    return(trend,dayssincecutover,trendnature)

In [16]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0}))
equityListdf['trend']=0
equityListdf['daysSinceCutOver']=0
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
equityListdf['RSI']=0
equityListdf['Close']=0
equityListdf['sharesbought']=0
equityListdf['moneyinvested']=0    
equityListdf['profit']=0
equityListdf['Capital']=0
equityListdf['trades']=0
equityListdf['cutoverprice']=0                
equityListdf['OBV']=0
capital_needed=0
equityListdf['sharesbought']=0
portfolio=np.array([['Symbol','price','number of shares']])
#print(equityListdf.columns)
profit=0  
trades=0
sell=0
buy=0
limit=20000
for symbol in equityListdf['Issuer Name']:
        
        # Check if database has latest sttock info by comparing aginst SBI
    for dt1 in stockWarehouse.find({'Symbol':'SBIN'},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
        refdate=dt1 # Set Reference Data as last date when SBI Stock Data was update
        for maxdate in stockWarehouse.find({'Symbol':symbol},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            if maxdate == refdate :  
            
                ################## Get Stock info for last 90 days########################
                datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol}).sort('Date',-1).limit(90))
                datadf.set_index('Date',inplace=True)
                datadf.sort_index(inplace=True)

                ##################Add Last close price to equityListdf
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Close']=datadf.iloc[-1,1]
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'%Deliverable']=datadf.iloc[-1,0]
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'High']=datadf.iloc[-1,4]
                tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy() # Conveting to numpy to use in if conditions
                if tempnp[0,6] < 1500 and tempnp[0,6] > 100 and tempnp[0,14] > .50:  # Column 6 is close price 
                    (a,b,c)=MovingAverageAnalysis(datadf,9,21)
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'trend']=a
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'daysSinceCutOver']=b
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'trendnature']=c
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI']=taLib.RSI(datadf['Close'],14)[-1]
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'OBV']=taLib.OBV(datadf['Close'],datadf['Volume'])[-1]
                    tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
                    if capital_needed < limit and tempnp[0,5] > 50 and tempnp[0,2] < 5   and tempnp[0,1] == 1 and tempnp[0,3] == 'Expanding' and tempnp[0,7] == 0 and tempnp[0,13] > 0:
                        equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Signal']=1
                        print(tempnp[0,14])
                    else:
                        if (tempnp[0,5] < 45 or tempnp[0,1] == -1 or tempnp[0,3] == 'Contracting') :
                            equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Signal']=-1

equityListdf.loc[equityListdf['Signal']>0].to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\Buy.csv")
equityListdf.loc[equityListdf['Signal']<0].to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\sell.csv")
equityListdf.to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\result.csv")
print("Buy")


                    
                   

0.5853
0.6443000000000001
0.5868
0.629
0.7044
0.6807
0.5552
0.7079000000000001
Buy


In [ ]:
print(equityListdf.columns)